# Infrascan Area Scanning Tool

This file allows an entire area of geocodes to by analyzed.
The score of each geocode is saved and a heatmap of the geocodes is generated.

In [1]:
import urllib
import requests
import json
import math
import msgpack
import random
import numpy as np
import matplotlib.pyplot as plt
from staticmap import StaticMap, CircleMarker
from multiprocessing import Pool

from constants import *
import overpass

In [2]:
with open("./research_data/final_geocodes.msgpack", "rb") as f:
    geocodes = msgpack.load(f)
    formatted_geocodes = [(g["g"]["lat"], g["g"]["lng"]) for g in geocodes]

In [ ]:
# this doesnt work anymore for some reason
# multithreading is broken and it takes days to run without it

# multithreading is very finicky,
# may have to run multiple times
processes_pool = Pool(5)

analyzed = []
num = 0

def threaded_analysis(geocode):
    anal = overpass.analyze_geocode(geocode)
    anal["geocode"] = geocode
    analyzed.append(anal)
    num+=1
    print(num, "/", len(formatted_geocodes))
    return anal

analyzed = processes_pool.map(threaded_analysis, formatted_geocodes)

In [15]:
scores = sorted(analyzed, key=lambda x: x["score"])
with open("./research_data/new_network_score.msgpack", "wb") as f:
    msgpack.dump(scores, f)

In [16]:
m = StaticMap(1000, 1000)

def color(score, alpha):
    dist = (100-abs((100-score) - (score)))/100 + 0.5
    return f"rgba({int(max((100-score)/100*255*dist, 0))}, {int(max((score-20)/100*255*dist, 0))}, 0, {alpha})"

# compute the average geocode
avg_geocode = (0, 0)
for score in scores:
    avg_geocode = (avg_geocode[0] + score["geocode"][0], avg_geocode[1] + score["geocode"][1])
avg_geocode = (avg_geocode[0]/len(scores), avg_geocode[1]/len(scores))

# draw each geocode's results as a colored dot
for score in scores:
    # skip geocodes more that 15km away from the average location
    if overpass.geo_distance(avg_geocode, score["geocode"]) > 15: continue
    for i in range(10):
        marker = CircleMarker((score["geocode"][1]+(random.random()*2-1)/200, score["geocode"][0]+(random.random()*2-1)/200), color(score["score"], 200), 20)
        m.add_marker(marker)
    marker = CircleMarker((score["geocode"][1], score["geocode"][0]), color(score["score"], 200), 40)
    m.add_marker(marker)

image = m.render(zoom=12)
image.save('marker.png')

ZeroDivisionError: division by zero